In [19]:
#si-exercise

import random
import numpy as np
import multiprocessing as mp
import timeit

#define constants
n_REPS = 1000
n_SAMPLES = 1000

#sample normal distributions and assign to variables
def generate_data(n_samples=n_SAMPLES):
  a = np.random.normal(15, 2, n_samples)
  x1 = np.random.normal(3, 5, n_samples)
  x2 = np.random.normal(10, 1, n_samples)
  x3 = np.random.normal(8, 8, n_samples)
  e = np.random.normal(0, 1, n_samples)
  y = a + x1 + 2 * x2 + 1/2 * x3 + e
  return y

#Serial calculation
def sCalc(n_reps=n_REPS):
  results = []
  for _ in range(n_reps):
    mean_y = generate_data().mean()
    results.append(mean_y)
  return results

#Parallel calculation
def pCalc(n_reps=n_REPS):
  results = []
  with mp.Pool(processes=2) as pool:
    for _ in range(n_reps):
      y = pool.apply_async(generate_data)
      mean_y = y.get().mean()
      results.append(mean_y)
  return results

#store sCalc results
sResults = sCalc(n_reps=n_REPS)
print(sResults)
sResults

#store pCalc results
pResults = pCalc(n_reps=n_REPS)
print(pResults)
pResults

#time sCalc
sTime = []
sTime.append(timeit.Timer('sCalc(n_reps=n_REPS)',
    'from __main__ import sCalc, generate_data, n_REPS').timeit(number=1))
print(sTime)
sTime

#time pCalc
pTime = []
pTime.append(timeit.Timer('pCalc(n_reps=n_REPS)',
    'from __main__ import pCalc, generate_data, n_REPS').timeit(number=1))
print(pTime)
pTime

[42.06264586683322, 41.8073639766884, 42.23133799084748, 42.06460353891675, 42.19558089106604, 42.23454439106744, 41.908441858426684, 42.13706528735236, 41.950695717635945, 42.078124689220914, 41.88081691661561, 41.95933702420023, 42.04802390208316, 42.021879335393514, 41.82112145458603, 41.971768790621304, 42.27970591364199, 42.18292826869729, 41.92079964436739, 41.98541391508679, 41.76358912656755, 41.93148857661216, 41.8919758916712, 42.04402440076891, 42.23971160080565, 42.19141961464284, 41.56699309938331, 42.18013163573104, 41.97650507970824, 41.78788312251331, 41.91070363152911, 42.192381054341965, 42.038494291777155, 41.77735582908547, 41.79489501302704, 42.20245448696128, 41.986656859468376, 41.94066538354503, 42.026002428601586, 42.205172250964885, 41.77359463728074, 42.51028337076797, 41.94253498941135, 42.23591056986072, 41.85146513398269, 42.04015810346088, 41.95450054356357, 41.98465296645349, 41.925824971219676, 42.08148674620075, 42.066226885458235, 42.15399709207947, 4

In [20]:
sResults

[42.06264586683322,
 41.8073639766884,
 42.23133799084748,
 42.06460353891675,
 42.19558089106604,
 42.23454439106744,
 41.908441858426684,
 42.13706528735236,
 41.950695717635945,
 42.078124689220914,
 41.88081691661561,
 41.95933702420023,
 42.04802390208316,
 42.021879335393514,
 41.82112145458603,
 41.971768790621304,
 42.27970591364199,
 42.18292826869729,
 41.92079964436739,
 41.98541391508679,
 41.76358912656755,
 41.93148857661216,
 41.8919758916712,
 42.04402440076891,
 42.23971160080565,
 42.19141961464284,
 41.56699309938331,
 42.18013163573104,
 41.97650507970824,
 41.78788312251331,
 41.91070363152911,
 42.192381054341965,
 42.038494291777155,
 41.77735582908547,
 41.79489501302704,
 42.20245448696128,
 41.986656859468376,
 41.94066538354503,
 42.026002428601586,
 42.205172250964885,
 41.77359463728074,
 42.51028337076797,
 41.94253498941135,
 42.23591056986072,
 41.85146513398269,
 42.04015810346088,
 41.95450054356357,
 41.98465296645349,
 41.925824971219676,
 42.0814867